In [2]:
"""
Memory
- 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
- 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
- 항상 세 개의 이모티콘으로 답장하도록 `FewShotPromptTemplate` 또는 `FewShotChatMessagePromptTemplate`을 사용하여 체인에 예시를 제공하세요.
- 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.
"""

content='감독: 김한민\n주요 출연진: 최민식, 조진웅, 류승룡\n예산: ₩25,000,000,000\n흥행 수익: ₩135,700,000,000\n장르: 역사, 전쟁, 액션\n시놉시스: 조선시대 임진왜란 당시 조선 해군의 최대 해전인 명량 해전을 다룬 역사 영화'
